<a href="https://colab.research.google.com/github/jessicayuan1/Insight-Lab/blob/main/Failure_Analysis_in_Power_Transformers_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
!pip install -q kaggle-cli

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
!kaggle datasets download shashwatwork/failure-analysis-in-power-transformers-dataset

kaggle.json
Dataset URL: https://www.kaggle.com/datasets/shashwatwork/failure-analysis-in-power-transformers-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/9.59k [00:00<?, ?B/s]
100% 9.59k/9.59k [00:00<00:00, 20.3MB/s]


In [ ]:
!pip install -q kaggle
!pip install -q kaggle-cli
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
!kaggle datasets download shashwatwork/failure-analysis-in-power-transformers-dataset

import os
import zipfile
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


zip_file = "failure-analysis-in-power-transformers-dataset.zip"
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("dataset")

csv_file = os.path.join("dataset", "Health index1.csv")
df = pd.read_csv(csv_file)
print("Dataset shape:", df.shape)
print(df.head())

df.fillna(df.mean(), inplace=True)

if 'Failure' not in df.columns:
    raise ValueError("Expected target column 'Failure' not found in dataset.")

X = df.drop(columns=['Failure'])
y = df['Failure']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

predictions = model.predict(X_test)
predictions_binary = (predictions > 0.5).astype(int)
print("Sample Predictions:", predictions_binary[:5])